In [ ]:
import model_config
import torch
import os

max_e = 150 #####  task idxs
sample_number = 20    ## number of samples used to compute AWCH
train_size =2000 ## number of training samples per class

config = model_config.set_config('none',test_size = 1000, train_size = train_size, max_epoch=max_e)
config['add_regulization'] = False
config['beta'] = 10
config['lss_fn'] = 'cse'
config['dataset'] = 'cifar10'  # 'mnist'  #'cifar10'  
config['model'] = 'MLP' # 'CNN'  #'FC' #'CNN' 
config['sample_holder'] = [0,1,2,3,4,5,6,7,8,9]  ## the samples used to compute AWCH
config['class_number'] = 10
config['B'] = 100 # 30
config['alpha'] = 0.1 # 0.5
net_size = 1000  

loss_fn = config['lss_fn']
sample_num  = config['train_size']
sample_holder = config['sample_holder']
mis_label_prob = config['rho']

batch_size = config['B']
# config['sample_holder'] = []
# config['class_number'] = 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
load_path = f"./AWCH_data/TrainSize{train_size}_SampleN{sample_number}_ClassN{len(config['sample_holder'])}_B{config['B']}lr{config['alpha']}_lossfn_{config['lss_fn']}_model_{config['model']}_dataset_{config['dataset']}/h_g_tensors_holder_epoch_{config['max_epoch']}.pt"
load_path = f"./AWCH_data/NS{net_size}_TrainSize{train_size}_SampleN{sample_number}_ClassN{len(config['sample_holder'])}_B{config['B']}lr{config['alpha']}_lossfn_{config['lss_fn']}_model_{config['model']}_dataset_{config['dataset']}/h_g_tensors_holder_epoch_{config['max_epoch']}.pt"
print(f"Loading data from: {load_path}")
loaded_data = torch.load(load_path,weights_only=False)
h_holder= loaded_data['h_holder']
g_holder= loaded_data['g_holder']
components= loaded_data['components']
Covar = loaded_data['Covar']
Hessian = loaded_data['matrix']
del loaded_data
components = torch.tensor(components).to(device)
h_holder = torch.tensor(h_holder).to(device)
h_holder = components @ h_holder @ components.T


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.ticker as ticker
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def set_icml_appendix_style():

    plt.rcParams.update({

        'font.family': 'serif',
        'font.serif': ['Times New Roman', 'Times', 'DejaVu Serif'],
        'mathtext.fontset': 'stix', 
        
        'font.size': 18,
        'axes.labelsize': 20,
        'axes.titlesize': 22,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 16,
        

        'lines.linewidth': 3.0,
        'lines.markersize': 8,
        'axes.linewidth': 2.0,
        

        'xtick.major.size': 8,
        'xtick.major.width': 1.5,
        'ytick.major.size': 8,
        'ytick.major.width': 1.5,

        'axes.grid': True,
        'grid.alpha': 0.4,
        'grid.linestyle': '--',
        'grid.linewidth': 1.0,

        'savefig.dpi': 300,
        'pdf.fonttype': 42,
        'ps.fonttype': 42
    })

def format_boxed_ax(ax):

    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(2.0)
        spine.set_color('black')
    ax.tick_params(direction='in', width=1.5, length=6, colors='black', grid_alpha=0.4)



def get_top_k_decomposition(h_holder, k=800):
    print(f"1. Performing Eigen Decomposition (Full Rank)...")
    h_holder = h_holder.to(device)
    vals, vecs = torch.linalg.eigh(h_holder)
    vals = torch.flip(vals, dims=[1])
    vecs = torch.flip(vecs, dims=[2])
    N = vals.shape[1]
    actual_k = min(k, N)
    print(f"2. Filtering Top {actual_k} modes...")
    vals_k = vals[:, :actual_k]
    vecs_k = vecs[:, :, :actual_k]
    del vals, vecs
    torch.cuda.empty_cache()
    return vals_k, vecs_k

def compute_H_C_only(eigenvalues, eigenvectors, chunk_size=16):
    N_samples = eigenvalues.shape[0]
    N_dim = eigenvectors.shape[1]
    H_mean_accumulator = np.zeros(N_dim, dtype=np.float32)
    C_mean_accumulator = np.zeros(N_dim, dtype=np.float32)

    with torch.no_grad():
        for i in range(0, N_samples, chunk_size):
            end_idx = min(i + chunk_size, N_samples)
            lam_chunk = eigenvalues[i:end_idx]
            vec_chunk = eigenvectors[i:end_idx]
            vec_chunk = torch.flip(vec_chunk, dims=[1]) 
            V_sq_chunk = vec_chunk ** 2
            
            H_batch_chunk = torch.matmul(V_sq_chunk, lam_chunk.unsqueeze(-1)).squeeze(-1)
            lam_sq_chunk = lam_chunk ** 2
            C_batch_chunk = torch.matmul(V_sq_chunk, lam_sq_chunk.unsqueeze(-1)).squeeze(-1)

            H_mean_accumulator += torch.sum(H_batch_chunk, dim=0).cpu().numpy()
            C_mean_accumulator += torch.sum(C_batch_chunk, dim=0).cpu().numpy()
            
            del lam_chunk, vec_chunk, V_sq_chunk, H_batch_chunk, C_batch_chunk
            
    return H_mean_accumulator / N_samples, C_mean_accumulator / N_samples

# def compute_full_gap_matrix_efficient(eigenvalues, eigenvectors, batch_size=32):
#     print(f"Computing Gap Matrix (Batch Size={batch_size})...")
#     B, N, num_modes = eigenvectors.shape 
#     eigenvectors = torch.flip(eigenvectors, dims=[1])
#     total_gap_sum = torch.zeros((N, num_modes), device=device)
#     with torch.no_grad():
#         for b_start in range(0, B, batch_size):
#             b_end = min(b_start + batch_size, B)
#             vec_chunk = eigenvectors[b_start:b_end] 
#             val_chunk = eigenvalues[b_start:b_end]      
#             lam_sq = (val_chunk ** 2).unsqueeze(1)
#             V_sq = vec_chunk ** 2
#             numerator = torch.cumsum(lam_sq * V_sq, dim=2) 
#             total_energy = numerator[:, :, -1].unsqueeze(2)
#             denominator = total_energy 
#             gap_chunk = numerator / denominator 
#             total_gap_sum += torch.sum(gap_chunk, dim=0)
#             del vec_chunk, val_chunk, lam_sq, V_sq, numerator, denominator, gap_chunk
#             torch.cuda.empty_cache()
        
#     avg_gap_matrix = total_gap_sum / B
#     return avg_gap_matrix.cpu().numpy()


def compute_full_gap_matrix_efficient(eigenvalues, eigenvectors, batch_size=32):
    print(f"Computing Gap Matrix (High Precision, Batch Size={batch_size})...")
    B, N, num_modes = eigenvectors.shape 
    eigenvectors = torch.flip(eigenvectors, dims=[1])
    
    total_gap_sum = torch.zeros((N, num_modes), device=device, dtype=torch.float64)
    
    with torch.no_grad():
        for b_start in range(0, B, batch_size):
            b_end = min(b_start + batch_size, B)
            
            vec_chunk = eigenvectors[b_start:b_end].to(torch.float64) 
            val_chunk = eigenvalues[b_start:b_end].to(torch.float64)
            
            lam_sq = (val_chunk ** 2).unsqueeze(1)
            V_sq = vec_chunk ** 2
            
            energy_contribution = lam_sq * V_sq
            numerator = torch.cumsum(energy_contribution, dim=2) 
            
            total_energy = numerator[:, :, -1].unsqueeze(2)
            
            gap_chunk = numerator / (total_energy) 
            
            total_gap_sum += torch.sum(gap_chunk, dim=0)
            
            del vec_chunk, val_chunk, lam_sq, V_sq, numerator, total_energy, gap_chunk
            torch.cuda.empty_cache()
        
    avg_gap_matrix = total_gap_sum / B
    return avg_gap_matrix.cpu().numpy().astype(np.float32)




def create_gap_from_real_spectrum(eigenvalues, idx = 10):

    gap_spectrum = eigenvalues.clone()
    gap_spectrum[:, idx:] = gap_spectrum[:, idx:] * 1e-5
    return gap_spectrum

def create_stiff_mean_spectrum(eigenvalues, idx=1, threshold_ratio=0.1, meancollapse=False):

    new_spectrum = eigenvalues.clone()
    new_spectrum[:, idx:] = new_spectrum[:, idx:] * 1e-5
    

    lam_0 = new_spectrum[:, 0]

    threshold = torch.max(lam_0) * threshold_ratio
    
    stiff_mask = lam_0 > threshold
    non_stiff_mask = ~stiff_mask

    if stiff_mask.sum() > 0 and meancollapse:
        stiff_mean = torch.mean(lam_0[stiff_mask])
        print(f"   [Stiff Analysis] Threshold: {threshold:.4f}, Count: {stiff_mask.sum().item()}/{len(lam_0)}, Mean Val: {stiff_mean:.4f}")

        new_spectrum[stiff_mask, 0] = stiff_mean
    if non_stiff_mask.sum() > 0:

        print(f"   [Stiff Analysis] Suppressing {non_stiff_mask.sum().item()} Non-Stiff samples by 1e-5.")
        new_spectrum[non_stiff_mask, 0] = new_spectrum[non_stiff_mask, 0] * 1e-3
        
    return new_spectrum



def plot_comparison_loglog(ax, data_orig, data_new, label_name, CH_slice):
    format_boxed_ax(ax)
    x = data_orig[CH_slice]
    y = data_new[CH_slice]
    
    mask = (x > 0) & (y > 0)
    x_clean, y_clean = x[mask], y[mask]

    if len(x_clean) > 1:
        log_x, log_y = np.log10(x_clean), np.log10(y_clean)
        slope, intercept, r_val, _, _ = linregress(log_x, log_y)
        
        ax.scatter(x_clean, y_clean, s=3, facecolors='none', edgecolors='#1f77b4', 
                   alpha=0.7, linewidth=0.5, label='Data Points')
        
        fit_y = 10**(intercept + slope * log_x)
        ax.plot(x_clean, fit_y, color='#d62728', linestyle='-', linewidth=1.0, label=f'Fit: slope={slope:.2f}')
        
        min_val = min(x_clean.min(), y_clean.min())
        max_val = max(x_clean.max(), y_clean.max())
        ref_line = np.linspace(min_val, max_val, 100)
        ax.plot(ref_line, ref_line, 'k--', linewidth=2.0, alpha=0.6, label='y=x')
    
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(f'Original {label_name}')
    ax.set_ylabel(f'Current {label_name}')
    ax.set_title(f'{label_name} Comparison')
    ax.legend(loc='lower right', frameon=True, framealpha=0.9, edgecolor='black', fancybox=False)


def analyze_slope_and_plot(CH_slice, H, C, gap_matrix, spectrum_data, row_title, 
                           H_ref, C_ref, 
                           ax_scatter, ax_gap, ax_spectrum, ax_comp_c, ax_comp_h,
                           end_idx=800):
    
    # --- 1. C_ii vs H_ii Scatter ---
    format_boxed_ax(ax_scatter)
    x_data = H[CH_slice]
    y_data = C[CH_slice]
    mask = (x_data > 0) & (y_data > 0)
    x_data, y_data = x_data[mask], y_data[mask]
    
    if len(x_data) > 1:
        log_x, log_y = np.log10(x_data), np.log10(y_data)
        slope, intercept, _, _, _ = linregress(log_x, log_y)
        ax_scatter.scatter(x_data, y_data, s=3, alpha=0.6, color='#000080', label='Data', edgecolors='none')
        fit_line_y = 10**(intercept + slope * log_x)
        ax_scatter.plot(x_data, fit_line_y, color='#d62728', linestyle='-', linewidth=1.0, label=f'Fit: {slope:.2f}')
        
        mid_x, mid_y = np.mean(log_x), np.mean(log_y)
        ref_line_x = np.logspace(min(log_x), max(log_x), 100)
        ref_line_y1 = 10**(1.0 * (np.log10(ref_line_x) - mid_x) + mid_y)
        ax_scatter.plot(ref_line_x, ref_line_y1, 'k--', linewidth=2.0, label='Slope=1')
        ref_line_y2 = 10**(2.0 * (np.log10(ref_line_x) - mid_x) + mid_y)
        ax_scatter.plot(ref_line_x, ref_line_y2, color='green', linestyle=':', linewidth=3.0, label='Slope=2')
    
    ax_scatter.set_xscale('log')
    ax_scatter.set_yscale('log')
    ax_scatter.set_xlabel(r'$H_{ii}$')
    ax_scatter.set_ylabel(r'$C_{ii}$')
    ax_scatter.set_title(f"Scalability ({row_title})", fontweight='bold') 
    ax_scatter.legend(loc='upper left', frameon=True, edgecolor='black', fancybox=False)
    
    # --- 2. Gap Curve ---
    format_boxed_ax(ax_gap)
    real_k = gap_matrix.shape[1]
    k_values = np.arange(1, real_k + 1)
    avg_gap_curve = np.mean(gap_matrix[:1500], axis=0)
    
    indices_to_plot = [1, 10, 50, 100, 400, 1000]
    colors_gap = plt.cm.viridis(np.linspace(0, 0.9, len(indices_to_plot))) 
    for idx, color in zip(indices_to_plot, colors_gap):
        if idx < gap_matrix.shape[0]:
            ax_gap.semilogx(k_values, gap_matrix[idx, :], color=color, linewidth=2.0, alpha=0.8, label=f'i={idx+1}')
    ax_gap.semilogx(k_values, avg_gap_curve, 'k--', linewidth=3.5, alpha=1.0, label='Avg')
    ax_gap.legend(loc='lower right', ncol=2, frameon=True, edgecolor='black', fancybox=False) 
    ax_gap.set_title("Gap Metric")
    ax_gap.set_xlabel('k (Cutoff)')
    ax_gap.set_ylabel('Cum. Distribution')
    ax_gap.set_ylim(-0.05, 1.1)
    
    # --- 3. Spectrum Plot ---
    format_boxed_ax(ax_spectrum)
    if isinstance(spectrum_data, torch.Tensor):
        spec_np = spectrum_data.detach().cpu().numpy()
    else:
        spec_np = spectrum_data
    B_samples = spec_np.shape[0]
    num_classes = 8
    colors = plt.cm.Set1(np.linspace(0, 1, num_classes))
    ranks = np.arange(1, spec_np.shape[1] + 1)
    selected_indices = np.linspace(0, B_samples-1, num_classes, dtype=int)
    for i, idx in enumerate(selected_indices):
        ax_spectrum.loglog(ranks, spec_np[idx], color=colors[i], alpha=0.8, linewidth=2.0)
    ax_spectrum.axvline(x=10, color='gray', linestyle=':', linewidth=2.5)
    ax_spectrum.set_title('Spectrum (Sampled)')
    ax_spectrum.set_xlabel('Rank')
    ax_spectrum.set_ylabel(r'$|\lambda_i|$')

    # --- 4. Comparison C ---
    plot_comparison_loglog(ax_comp_c, C_ref, C, r'$C_{ii}$', CH_slice)

    # --- 5. Comparison H ---
    plot_comparison_loglog(ax_comp_h, H_ref, H, r'$H_{ii}$', CH_slice)


def main(CH_slice, h_holder_input=None, fig_title="Neural Collapse Spectral Analysis"):
    set_icml_appendix_style()
    h_holder = h_holder_input
    B, N, _ = h_holder.shape
    print(f"Processing input data: B={B}, N={N}")

    # Baseline: Real Data
    torch.backends.cuda.preferred_linalg_library('magma')
    k_target_actual = min(N, 2000) 
    real_eigenvalues_orig, eigenvectors = get_top_k_decomposition(h_holder, k=k_target_actual)
    del h_holder 
    torch.cuda.empty_cache()

    results = []
    
    # 1. Original
    print("\n--- Processing Real Data (Original) ---")
    H_r_orig, C_r_orig = compute_H_C_only(real_eigenvalues_orig, eigenvectors)
    Gap_r_orig = compute_full_gap_matrix_efficient(real_eigenvalues_orig, eigenvectors)
    results.append({'H': H_r_orig, 'C': C_r_orig, 'Gap': Gap_r_orig, 'Spec': real_eigenvalues_orig, 'Title': "Original"})
    
    # 2. Cut @ k=1 (Tail Compression)
    print("\n--- Processing Real Data (Cut 1) ---")
    real_eigenvalues_smooth2 = create_gap_from_real_spectrum(real_eigenvalues_orig, idx=1)
    H_r_smooth2, C_r_smooth2 = compute_H_C_only(real_eigenvalues_smooth2, eigenvectors)
    Gap_r_smooth2 = compute_full_gap_matrix_efficient(real_eigenvalues_smooth2, eigenvectors)
    results.append({'H': H_r_smooth2, 'C': C_r_smooth2, 'Gap': Gap_r_smooth2, 'Spec': real_eigenvalues_smooth2, 'Title': f"Cut @ k=1"})

    # 3. Cut @ k=1 & Stiff Mean Collapse (MODIFIED)
    print("\n--- Processing Real Data (Stiff Mean Collapse) ---")

    real_eigenvalues_smooth3 = create_stiff_mean_spectrum(real_eigenvalues_orig, idx=1, threshold_ratio=0.001, meancollapse=False)
    
    H_r_smooth3, C_r_smooth3 = compute_H_C_only(real_eigenvalues_smooth3, eigenvectors)
    Gap_r_smooth3 = compute_full_gap_matrix_efficient(real_eigenvalues_smooth3, eigenvectors)
    results.append({'H': H_r_smooth3, 'C': C_r_smooth3, 'Gap': Gap_r_smooth3, 'Spec': real_eigenvalues_smooth3, 'Title': f"Cut @ k=1 & Stiff Preserve"})


    # 4. Cut @ k=10 & Stiff Mean Collapse (MODIFIED)
    print("\n--- Processing Real Data (Cut 10 & Stiff Mean Collapse) ---")
    real_eigenvalues_smooth4 = create_stiff_mean_spectrum(real_eigenvalues_orig, idx=1, threshold_ratio=0.001,meancollapse=True)
    H_r_smooth4, C_r_smooth4 = compute_H_C_only(real_eigenvalues_smooth4, eigenvectors)
    Gap_r_smooth4 = compute_full_gap_matrix_efficient(real_eigenvalues_smooth4, eigenvectors)
    results.append({'H': H_r_smooth4, 'C': C_r_smooth4, 'Gap': Gap_r_smooth4, 'Spec': real_eigenvalues_smooth4, 'Title': f"Cut @ k=1 & Stiff Mean Collapse"})

    # Plotting
    total_rows = len(results)
    print(f"\nGenerating Plots...")
    fig, axes = plt.subplots(total_rows, 5, figsize=(24, 5.0 * total_rows), constrained_layout=True)
    if total_rows == 1: axes = np.expand_dims(axes, 0)

    for i, res in enumerate(results):
        analyze_slope_and_plot(CH_slice,
            res['H'], res['C'], res['Gap'], res['Spec'], res['Title'],
            H_r_orig, C_r_orig, 
            axes[i, 0], axes[i, 1], axes[i, 2], axes[i, 3], axes[i, 4],
            end_idx=k_target_actual)
    
    fig.suptitle(fig_title, fontsize=28, fontweight='bold', fontfamily='serif')
    

    save_dir = "ICML_Figures/Ablation"
    os.makedirs(save_dir, exist_ok=True)
    filename = f"Spectrum_stiff_mean_ablation.pdf"
    save_path = os.path.join(save_dir, filename)
    print(f"Saving to {save_path} ...")
    plt.savefig(save_path, format='pdf', bbox_inches='tight', pad_inches=0.2, dpi=300) 
    plt.show()
    print("Done.")

if __name__ == "__main__":
    if 'h_holder' in locals():
        MY_TITLE = "Effect of Stiff Sample Homogenization"
        CH_slice = torch.arange(0, 1500)
        main(CH_slice, h_holder, fig_title=MY_TITLE)
    else:
        print("Wait: 'h_holder' is not defined.")